# **Limpieza**

#### LIBRERÍAS

In [1]:
import pandas as pd # Manipulación df
import os # Directorios
import re # Manipulación de cadenas de texto (expresiones regulares)
import unicodedata# Función para eliminar acentos
from datetime import datetime, date  # Para trabajar con fechas y horas
from currency_converter import CurrencyConverter # convertidor de monedas
from dateutil.relativedelta import relativedelta  # Para calcular diferencias entre fechas, incluyendo años, meses y días

#### CARGAR ARCHIVO

Solo cambia el nombre del archivo en 'csv_filename' y su ruta con 'csv_dir'

In [2]:
directorio_actual = os.getcwd() # Directorio actual de trabajo
# Ruta relativa al archivo CSV
csv_filename = 'lamudi-Abril-QueretaroInmuebles .xlsx'
csv_dir = os.path.join(directorio_actual)#, 'datos'
csv_path = os.path.join(csv_dir, csv_filename)

Elimina el # de la primer línea si el archivo a leer no contiene el nombre de las columnas

In [4]:
#column_names=['precio','denominacion','propiedad','metros_total','metros_construido','tiempo_de_publicacion','tipo','estacionamientos','recamaras','banos','medio_banos','seguridad_privada','ubicacion','url','descripcion']
df = pd.read_excel(csv_path)#, names=column_names
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9344 entries, 0 to 9343
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   precio                 9344 non-null   int64 
 1   denominacion           9344 non-null   object
 2   propiedad              9344 non-null   object
 3   metros_total           9344 non-null   int64 
 4   metros_construido      9344 non-null   int64 
 5   tiempo_de_publicacion  9344 non-null   object
 6   tipo                   9343 non-null   object
 7   estacionamientos       9344 non-null   int64 
 8   recamaras              9344 non-null   int64 
 9   banos                  9344 non-null   int64 
 10  medio_banos            9344 non-null   int64 
 11  seguridad_privada      9344 non-null   object
 12  ubicacion              9344 non-null   object
 13  url                    9344 non-null   object
 14  descripcion            9344 non-null   object
dtypes: int64(7), object(8

#### LIMPIEZA NULOS

In [4]:
df.fillna({'precio': 0}, inplace=True) 
df.fillna({'propiedad': 'Propiedad sin título'}, inplace=True) 
df.fillna({'metros_total': 0}, inplace=True) 
df.fillna({'metros_construido': 0}, inplace=True) 
df.fillna({'tiempo_de_publicacion': '1 ene 1000'}, inplace=True) 
df.fillna({'tipo': 'Propiedad sin tipo de casa'}, inplace=True) 
df.fillna({'estacionamientos': 0}, inplace=True) 
df.fillna({'recamaras': 0}, inplace=True) 
df.fillna({'banos': 0}, inplace=True) 
df.fillna({'medio_banos': 0}, inplace=True) 
df.fillna({'seguridad_privada': 'No'}, inplace=True) 
df.fillna({'ubicacion': 'Propiedad sin ubicación'}, inplace=True) 
df.fillna({'url': '-'}, inplace=True) 
df.fillna({'descripcion': "Propiedad sin descripcion"}, inplace=True) 

In [5]:
df.isna().sum()

precio                   0
denominacion             0
propiedad                0
metros_total             0
metros_construido        0
tiempo_de_publicacion    0
tipo                     0
estacionamientos         0
recamaras                0
banos                    0
medio_banos              0
seguridad_privada        0
ubicacion                0
url                      0
descripcion              0
dtype: int64

# 1. Manipulación de precios

## a) Precio en descripción

Agrega las diferentes formas de que el precio se encuentre en la descripción.

In [7]:
def extraer_precio(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones
    precios = re.findall(r'\b(\d+[\.,]?\d*[\.,]?\d*)\s*($|mil pesos|mxn|precio|pesos|monto|Pesos Mexicanos|USD)', descripcion)
    precio = None
    for match in precios:
        numero, moneda = match
        numero = numero.replace(',', '').replace("'", '').replace(".",'')
        try:
            precio = float(numero)
        except ValueError:
            continue
        if precio:
            break
    return precio

# Denominación de la descripción
def extraer_denominacion(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones 
    denominaciones = re.findall(r'\b(mil|mxn|precio|pesos|monto|Mexicanos|Estadounidenses)\b', descripcion)
    return denominaciones[0] if denominaciones else None

df['precio_desc'] = df['descripcion'].apply(extraer_precio)
#df['denominacion_desc'] = df['descripcion'].apply(extraer_denominacion)

### Precio [0 / 0]

In [8]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas
# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 o 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


Eliminar registros que no recuperaron su precio ni desde la descripción

In [9]:
df = df[~( (df['precio'] == 0 ) & (df['precio_desc'].isna()) )] 

### Precio [0 / #]

In [10]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['precio'] == 0 ) & (df['precio_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Precio [# / #]

In [11]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['precio']!= 0) & (df['precio_desc'].notna())) & (df['precio'] != df['precio_desc']) )] 
# Filtro dentro del estándar
totales = totales[(totales['precio_desc'] >= 250000) & (totales['precio_desc'] <= 500000000)]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['precio', 'precio_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'precio' por el de 'precio_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'precio': ")
        df.at[idx, 'precio'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'precio'] = df.at[idx, 'precio_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Registro 22:
precio                                                                                   600000
precio_desc                                                                           2500000.0
url            https://www.lamudi.com.mx//detalle/41032-73-6777bfd1c02c-5ecc-c4010eec-81e6-47d4
Name: 22, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'precio' por el de 'precio_desc'
3. Pasar al siguiente registro
Registro 23:
precio                                                                                   610000
precio_desc                                                                          61000000.0
url            https://www.lamudi.com.mx//detalle/41032-73-58c0cd447fd0-bf57-407329a9-b48b-429c
Name: 23, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'precio' por el de 'precio_desc'
3. Pasar al siguiente registro
Registro 54:
precio               

## b) Denominación MXN

SI EXISTE UNA DENOMINACIÓN QUE NO SEA MXN, SE CONVIERTA EL PRECIO

In [8]:
import datetime
c = CurrencyConverter() # Instancia del convertidor de monedas
def convertir_a_mxn(precio, denominacion):
    if denominacion != 'MXN':
        return int(round(c.convert(precio, denominacion, 'MXN')))
    return precio

def convertir_a_usd(precio, denominacion):
    if denominacion != 'USD':
        return round(c.convert(precio, denominacion, 'USD'), 2)
    return precio

# Obtener la fecha actual
fecha_conversion = datetime.datetime.now().date()

# Aplicar la conversión a cada fila del DataFrame y agregar la fecha de conversión
df['precio_mxn'] = df.apply(lambda row: convertir_a_mxn(row['precio'], row['denominacion']), axis=1)
df['precio_usd'] = df.apply(lambda row: convertir_a_usd(row['precio'], row['denominacion']), axis=1)
#df['denominacion_mxn'] = 'MXN'
#df['denominacion_usd'] = 'USD'
df['fecha_conversion'] = fecha_conversion

print(df[['precio', 'denominacion', 'precio_mxn', 'precio_usd', 'fecha_conversion']].head())

# Mover las columnas 'precio_mxn' y 'precio_usd' al inicio del DataFrame
#cols = ['precio_mxn', 'precio_usd'] + [col for col in df.columns if col not in ['precio_mxn', 'precio_usd']]
#df = df[cols]

    precio denominacion  precio_mxn  precio_usd fecha_conversion
0  2640000          MXN     2640000   146849.88       2024-09-09
1  6950000          MXN     6950000   386593.44       2024-09-09
2  2908782          MXN     2908782   161800.87       2024-09-09
3   280000          MXN      280000    15574.99       2024-09-09
4   400000          MXN      400000    22249.98       2024-09-09


## c) Precio fuera del estandar

IDENTIFICAR Y ELIMINAR PROPIEDADES QUE CUENTEN CON PRECIOS BAJOS y ALTOS AL ESTANDAR

In [9]:
# Eliminar los registros donde el precio es menor al estandar (250,000) y con precio_desc vacío
df = df[(df['precio'] >= 250000)] #& (df['precio_desc'].notna())]
# Eliminar los registros donde el precio es mayor al estandar (500,000,000) y con precio_desc vacío
df = df[(df['precio'] <= 500000000)] #& (df['precio_desc'].notna())] 
# Ordenar valores de manera ascendente
df['precio'].sort_values()

3         280000
4         400000
6         464000
7         490000
8         490000
          ...   
7209    52500000
9282    54800000
9283    54800000
9284    58000000
8127    76460000
Name: precio, Length: 9344, dtype: int64

## D) Segmento o categoría

In [10]:
bins = [0, 500000, 750000, 1000000, 1250000, 1500000, 1750000, 2000000, 2250000, 2500000,
        2750000, 3000000, 3250000, 3500000, 3750000, 4000000, 6000000, 8000000, 12000000,
        14000000, 16000000, 18000000, 22000000, float('inf')]
labels = ["E1", "E2", "E3", "D1", "D2", "D3", "C1", "C2", "C3", "B1", "B2", "B3",
          "A1", "A2", "A3", "S1", "S2", "S3", "L1", "L2", "L3", "L+", "ELITE"]

# Utilizar pd.cut para clasificar los valores
df['categoria'] = pd.cut(df['precio'], bins=bins, labels=labels, right=False)
print(df['categoria'].unique())

['B1', 'S2', 'B2', 'E1', 'E2', ..., 'L3', 'S3', 'L1', 'L+', 'L2']
Length: 23
Categories (23, object): ['E1' < 'E2' < 'E3' < 'D1' ... 'L2' < 'L3' < 'L+' < 'ELITE']


# 2. Meses transcurridos

Sacar el número de meses transcurridos desde la fecha de su publicación a la actual

In [11]:
df['tiempo_de_publicacion']

0       2024-03-01 00:00:00
1       2023-11-25 00:00:00
2              22 ago. 2023
3              20 abr. 2023
4               1 dic. 2023
               ...         
9339           30 ene. 2024
9340    2023-10-07 00:00:00
9341    2024-02-12 00:00:00
9342           24 abr. 2023
9343           24 abr. 2023
Name: tiempo_de_publicacion, Length: 9344, dtype: object

In [14]:
from datetime import datetime
# Convertir el mes abreviado en número
def month_to_int(month):
    months = ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']
    return months.index(month.lower()) + 1

# Estandarizar el formato de fecha
def standardize_date(date_str):
    # Caso 1: formato "20 abr. 2023" -> "2023-04-20"
    try:
        day, month, year = date_str.split(' ')
        month_int = month_to_int(month.replace('.', ''))  # Eliminar puntos si existen
        return f"{year}-{month_int:02d}-{int(day):02d}"
    except (ValueError, AttributeError):
        pass

    # Caso 2: formato "YYYY-MM-DD HH:MM:SS" o "YYYY-MM-DD"
    try:
        return pd.to_datetime(date_str).strftime('%Y-%m-%d')
    except (ValueError, TypeError):
        # Si no se puede convertir, devolver None
        return None

# Aplicar la función a la columna 'tiempo_de_publicacion'
df['fecha_estandarizada'] = df['tiempo_de_publicacion'].apply(standardize_date)

# Convertir a datetime y manejar errores
df['fecha_estandarizada'] = pd.to_datetime(df['fecha_estandarizada'], errors='coerce')

# Obtener la fecha actual
fecha_actual = datetime.now().date()

# Función para calcular la diferencia en meses
def calcular_meses(fecha):
    if pd.isnull(fecha):
        return None
    rd = relativedelta(fecha_actual, fecha.date())
    return rd.years * 12 + rd.months

# Aplicar la función para calcular los meses transcurridos
df['meses_transcurridos'] = df['fecha_estandarizada'].apply(calcular_meses)

# Mostrar el DataFrame con las fechas estandarizadas y meses transcurridos
print(df[['tiempo_de_publicacion', 'fecha_estandarizada', 'meses_transcurridos']])


     tiempo_de_publicacion fecha_estandarizada  meses_transcurridos
0      2024-03-01 00:00:00          2024-03-01                  6.0
1      2023-11-25 00:00:00          2023-11-25                  9.0
2             22 ago. 2023          2023-08-22                 12.0
3             20 abr. 2023          2023-04-20                 16.0
4              1 dic. 2023          2023-12-01                  9.0
...                    ...                 ...                  ...
9339          30 ene. 2024          2024-01-30                  7.0
9340   2023-10-07 00:00:00          2023-10-07                 11.0
9341   2024-02-12 00:00:00          2024-02-12                  6.0
9342          24 abr. 2023          2023-04-24                 16.0
9343          24 abr. 2023          2023-04-24                 16.0

[9344 rows x 3 columns]


# 3. Tipo de propiedad

## a) Lista de tipos deseados

In [15]:
# Filtrar los registros según los tipos especificados
tipos_permitidos = 'Casa|Casa en Fraccionamiento|Casa en Condominio|Departamento|Penthouse|Dúplex|Loft|Estudio|Condominio Horizontal'
df = df[df['tipo'].str.contains(tipos_permitidos, na=False)]
df['tipo'].unique()

array(['Casa', 'Departamento', 'Casa en Fraccionamiento', 'Dúplex',
       'Condominio Horizontal', 'Casa en Condominio'], dtype=object)

## b) Manejo de terrenos

Se buscan los registros que no sean las propiedades si no un solo terreno

In [33]:
# Filtrar los registros según las condiciones dadas
totales =df[df['descripcion'].str.contains('lotes', case=False)]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# MANEJAR PROCESO DE ELIMINACIÓN 
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])#'descripcion',
    
    print("\n¿Te gustaría eliminar el registro?")
    # Solicitar la opción al usuario
    option = input("Elige una opción (si / no): ").strip().lower()
    
    if option == 'si':
        df.drop(idx, inplace=True)
        print(f"Registro eliminado.\n")
    else:
        print(f"Registro conservado.\n")
        print("\n---\n")
print("Proceso completado.")

Registro 1368:
url    https://www.lamudi.com.mx//detalle/41032-73-9d044785283c-c487-3cbac49-8ab7-1237
Name: 1368, dtype: object

¿Te gustaría eliminar el registro?
Registro conservado.


---

Registro 2563:
url    https://www.lamudi.com.mx//detalle/41032-73-9143f2f0a9cf-e27d-abed963e-b9c9-3132
Name: 2563, dtype: object

¿Te gustaría eliminar el registro?
Registro conservado.


---

Registro 3093:
url    https://www.lamudi.com.mx//detalle/41032-73-4424109420fc-509f-71bc25aa-8c0a-3374
Name: 3093, dtype: object

¿Te gustaría eliminar el registro?
Registro conservado.


---

Registro 3621:
url    https://www.lamudi.com.mx//detalle/41032-73-2c97629f6a2e-db6c-6a4743c3-9c42-3a2e
Name: 3621, dtype: object

¿Te gustaría eliminar el registro?
Registro conservado.


---

Registro 4282:
url    https://www.lamudi.com.mx//detalle/41032-73-d3b5e34805e8-4bd5-d9244655-b7b8-3e0a
Name: 4282, dtype: object

¿Te gustaría eliminar el registro?
Registro conservado.


---

Registro 4436:
url    https://www.la

# 4. Metros totales

## a) Metros en descripción

In [34]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|totales|mt2)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['totales_desc'] = df['descripcion'].apply(extraer_totales)

### Metros [0 / 0]

In [35]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [0 / #]

In [36]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_total', 'totales_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_total': ")
        df.at[idx, 'metros_total'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [# / #]

In [38]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_total']!= 0) & (df['totales_desc'].notna())) & (df['metros_total'] != df['totales_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(f"metros_total: {row['metros_total']}")
    print(f"totales_desc: {row['totales_desc']}")
    print(f"url: {row['url']}")
    
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_total' por el de 'totales_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        while True:
            try:
                new_value = float(input("Introduce el nuevo valor para 'metros_total': "))
                df.at[idx, 'metros_total'] = new_value
                break
            except ValueError:
                print("Por favor, introduce un número válido (entero o decimal).")
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'metros_total'] = df.at[idx, 'totales_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    print("\n---\n")

print("Proceso completado.")

Registro 144:
metros_total: 98
totales_desc: 62.0
url: https://www.lamudi.com.mx//detalle/41032-73-92df250cdc82-e8da-5c83126a-9d82-3842

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Pasar al siguiente registro
Registro 170:
metros_total: 134
totales_desc: 0.0
url: https://www.lamudi.com.mx//detalle/41032-73-3e713fb07eab-9e59-ddbeb82f-aa33-4d40

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Pasar al siguiente registro
Registro 182:
metros_total: 130
totales_desc: 0.0
url: https://www.lamudi.com.mx//detalle/41032-73-dec2ffad4e4a-462f-8845403-852f-434d

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Pasar al siguiente registro
Registro 203:
metros_total: 105
totales_desc: 0.0
url: https://www.lamudi.com.mx/detalle/41032-73-

C:\Users\yoe11\AppData\Local\Temp\ipykernel_20860\929714313.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '515.18' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'metros_total'] = new_value



---

Registro 329:
metros_total: 155
totales_desc: 0.0
url: https://www.lamudi.com.mx//detalle/41032-73-4c63706e247b-9f0e-ed6c285f-88f9-4599

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Pasar al siguiente registro
Registro 386:
metros_total: 204
totales_desc: 0.0
url: https://www.lamudi.com.mx/detalle/41032-73-baa68ede8f38-2abd-645b23fe-b4e6-418f

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Pasar al siguiente registro
Registro 394:
metros_total: 79
totales_desc: 57.66
url: https://www.lamudi.com.mx//detalle/41032-73-e9436a301644-a2b2-72eab11c-a413-3f7a

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_total' por el de 'totales_desc'
3. Pasar al siguiente registro
Registro 497:
metros_total: 70
totales_desc: 7000.0
url: https://www.lamudi.com.mx//detalle

## b) Manejo por rangos de UMA

Paso opcional por si no se espera cambiar el precio por el que se tenga en descripción

Rangos
* 118 UMAS = 40m2
* 118.1-200 = 50m2
* 200.1-350 = 71m2
* 350.1-750 = 102m2
* 750.1-1500 = 156m2

### DF cuando ambas columnas son 0 o vacíos

In [ ]:
# Rangos de UMAs y sus equivalencias en m²
umas_to_m2 = [
    (118, 40),
    (200, 50),
    (350, 71),
    (750, 102),
    (1500, 156)
]
valor_uma = 108.57

# Convertir MXN a UMAs
def precio_a_umas(precio, valor_uma):
    return precio / valor_uma

# Asignar m² basado en el precio en UMAs
def asignar_m2(precio_umas):
    for upper_bound, m2 in umas_to_m2:
        if precio_umas <= upper_bound:
            return m2
    return None  # En caso de que el precio en UMAs sea mayor al máximo definido

# Actualizar los registros con 'metros_total' igual a 0 y 'totales_desc' es NaN
def actualizar_metros_total(row):
    if row['metros_total'] == 0 and pd.isna(row['totales_desc']):
        precio_umas = precio_a_umas(row['precio'], valor_uma)
        row['metros_total'] = asignar_m2(precio_umas)
    return row

totales = df[( (df['metros_total'] == 0 ) & (df['totales_desc'].isna() | df['totales_desc']==0) )] 
totales = df.apply(actualizar_metros_total, axis=1)
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

#df = df.apply(actualizar_metros_total, axis=1)
print(totales[['metros_total', 'totales_desc', 'url']])

      metros_total  totales_desc  \
6            100.0           NaN   
13           350.0           NaN   
18           500.0           NaN   
31           483.0           NaN   
37           277.0           NaN   
...            ...           ...   
9285         900.0           NaN   
9317         634.0           NaN   
9321        1047.0           NaN   
9327         750.0           NaN   
9332        1047.0           NaN   

                                                                                  url  
6     https://www.lamudi.com.mx/detalle/41032-73-7470e7c0f69b-8f03-1adcb58d-9a14-405d  
13    https://www.lamudi.com.mx/detalle/41032-73-bd5a86939b0e-2cb6-83bc8753-80aa-49c5  
18     https://www.lamudi.com.mx/detalle/41032-73-1db2d29a0f95-16c-34653316-be19-4b01  
31     https://www.lamudi.com.mx/detalle/41032-73-93d194dc33d-d035-bf07501c-825e-4a4a  
37    https://www.lamudi.com.mx/detalle/41032-73-27915b7692e9-3bc0-67bbfd01-9453-4608  
...                                    

In [ ]:
# Si se aceptan los cambios:
df = df.apply(actualizar_metros_total, axis=1)

# 5. M2 construidos

## a) M-construidos en descripción

In [39]:
# Función para extraer totales de la descripción
def extraer_totales(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(metros|construidos)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['construidos_desc'] = df['descripcion'].apply(extraer_totales)

### Metros [0 / 0]

In [40]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].isna() | df['construidos_desc']==0) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [0 / #]

In [41]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['metros_construido'] == 0 ) & (df['construidos_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        df.at[idx, 'metros_construido'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Metros [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['metros_construido']!= 0) & (df['construidos_desc'].notna())) & (df['metros_construido'] != df['construidos_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['metros_construido', 'construidos_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'metros_construido': ")
        try:
            df.at[idx, 'metros_construido'] = float(new_value)
        except ValueError:
            print("Valor no válido. Debe ser un número. Pasando al siguiente registro...")
    elif option == '2':
        # Sustituir el valor de 'metros_construido' por 'construidos_desc'
        df.at[idx, 'metros_construido'] = df.at[idx, 'construidos_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Registro 593:
metros_construido                                                                                123
construidos_desc                                                                                 0.0
url                  https://www.lamudi.com.mx/detalle/41032-73-85bc36bf8eaf-6efb-327a921b-859e-4c09
Name: 593, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'
3. Pasar al siguiente registro
Registro 849:
metros_construido                                                                                200
construidos_desc                                                                                14.0
url                  https://www.lamudi.com.mx/detalle/41032-73-9a66a6c61eb4-78df-71c0c7b7-bc27-3f1c
Name: 849, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'metros_construido' por el de 'construidos_desc'

# 6. Precio / m2

## M2 Totales (terreno)

In [16]:
# Calcular precio por metro cuadrado del terreno
df['precio_m2_terreno'] = df['precio']/df['metros_total']

## M2 Construidos

In [17]:
# Calcular precio por metro cuadrado de la cosntrucción
df['precio_m2_construido'] = df['precio']/df['metros_construido']

# 7. Estacionamientos

## a) Estacionamientos descritos

Asigna las diferentes maneras en las que puede encontrarse descrito el espacio de estacionamientos

In [18]:
# Función para extraer estacionamientos de la descripción
def extraer_coches(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(|cochera|garage|carros|carro|coche|coches|autos|automóviles|automóvil|auto|lugares de estacionamiento)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '').replace('.','')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['estacionamiento_desc'] = df['descripcion'].apply(extraer_coches)

### Estacionamientos [0 / 0]

In [19]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].isna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)    
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Estacionamientos [0 / #]

In [20]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['estacionamientos']==0) & (df['estacionamiento_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Estacionamientos [# / #]

In [21]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['estacionamientos']!=0) & (df['estacionamiento_desc'].notna())) & (df['estacionamientos'] != df['estacionamiento_desc']) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['estacionamientos', 'estacionamiento_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'estacionamientos': ")
        df.at[idx, 'estacionamientos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'estacionamientos'] = df.at[idx, 'estacionamiento_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Registro 0:
estacionamientos                                                        2
estacionamiento_desc                                               2890.0
url                     https://www.lamudi.com.mx//detalle/41032-73-1e...
Name: 0, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'
3. Pasar al siguiente registro


Registro 1:
estacionamientos                                                        2
estacionamiento_desc                                                 24.0
url                     https://www.lamudi.com.mx//detalle/41032-73-a6...
Name: 1, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'
3. Pasar al siguiente registro
Registro 4:
estacionamientos                                                        2
estacionamiento_desc                                                  3.0
url                     https://www.lamudi.com.mx/detalle/41032-73-9a6...
Name: 4, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'estacionamientos' por el de 'estacionamiento_desc'
3. Pasar al siguiente registro
Registro 5:
estacionamientos                                                        2
estacionamiento_desc                          

## b) Mantener rango de estacionamientos < 5 (mínimo 1)

In [ ]:
df_filtrado = df[df['estacionamientos'] <= 5]
df_filtrado[['estacionamientos','estacionamiento_desc','url']]

In [ ]:
df = df[df['estacionamientos'] <= 5]

# 8. Recámaras

## a) Recámaras en descripción

In [ ]:
# Función para extraer cantidad de recamaras en descripción
def extraer_recamaras(descripcion):
    descripcion = descripcion.lower()
    # Buscar patrones de totales
    totales = re.findall(r'\b(\d+[\.,]?\d*)\s*(recamara|recamaras|habitaciones|dormitorio|dormitorios)\b', descripcion)
    total = None
    for match in totales:
        numero, _ = match
        numero = numero.replace(',', '').replace("'", '')
        try:
            total = float(numero)
        except ValueError:
            continue
        if total:
            break
    return total

# Aplicar la función al DataFrame
df['recamaras_desc'] = df['descripcion'].apply(extraer_recamaras)

### Recámaras [0 / 0]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[ (df['recamaras'] == 0 ) & (df['recamaras_desc'].isna()) ] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")

Proceso completado.


### Recámaras [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']==0) & (df['recamaras_desc'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Proceso completado.


### Recámaras [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['recamaras']!=0) & (df['recamaras_desc'].notna()) ) & (df['recamaras'] != df['recamaras_desc'])] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['recamaras', 'recamaras_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'recamaras' por el de 'recamaras_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'recamaras': ")
        df.at[idx, 'recamaras'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'metros_total' por 'totales_desc'
        df.at[idx, 'recamaras'] = df.at[idx, 'recamaras_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


## b) Mantener rango de recámaras <= 7 

In [ ]:
df['recamaras'].unique()

In [ ]:
dentro = df[(df['recamaras'] <= 7)] 
print(dentro[['recamaras','recamaras_desc','url']])

# 9. Baños

## a) Baños en descripción

Manejando tanto los completos como los medios

In [ ]:
# Extraer la cantidad de baños y medios baños en la descripción
def extraer_baños(descripcion):
    descripcion = descripcion.lower()
    
    # Buscar patrones de baños completos
    completos = re.findall(r'\b(\d+[\.,]?\d*)\s*baños?\b', descripcion)
    total_completos = sum([float(match.replace(',', '').replace("'", '')) for match in completos])
    
    # Buscar patrones de medios baños (1/2 baño)
    medios = re.findall(r'\b(\d+[\.,]?\d*)\s*medios?\s*baños?\b', descripcion)
    total_medios = sum([float(match.replace(',', '').replace("'", '')) for match in medios])
    
    # Retornar como un diccionario o una tupla
    return {'completos': total_completos, 'medios': total_medios}

# Aplicar la función al DataFrame
df['baños_desc'] = df['descripcion'].apply(extraer_baños)

# Si deseas separar en dos columnas distintas:
df['baños_completos'] = df['baños_desc'].apply(lambda x: x['completos'] if x else 0)
df['medios_baños'] = df['baños_desc'].apply(lambda x: x['medios'] if x else 0)

### Baños

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con medio baño en descripción que se le asigne ese valor.

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['banos'] == 0) & (df['baños_completos'].isna())]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Proceso completado.


#### [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['banos']==0) & (df['baños_completos'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['banos']])
    print(row[['baños_completos']])
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Registro 236:
banos    0.0
Name: 236, dtype: object
baños_completos    0.0
Name: 236, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-a3ee8bf8fc9d-524a-1f3db509-83cd-4b7b
Name: 236, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro


KeyboardInterrupt: Interrupted by user

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['banos']!=0) & (df['baños_completos'].notna())) & (df['banos'] != df['baños_completos']))] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['banos']])
    print(row[['baños_completos']])    
    print(row[['url']])
    
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'banos' por el de 'baños_completos'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'banos' por 'baños_completos'
        df.at[idx, 'banos'] = df.at[idx, 'baños_completos']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

Registro 31:
banos    2
Name: 31, dtype: object
baños_completos    5.0
Name: 31, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-93d194dc33d-d035-bf07501c-825e-4a4a
Name: 31, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 37:
banos    3
Name: 37, dtype: object
baños_completos    0.0
Name: 37, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-27915b7692e9-3bc0-67bbfd01-9453-4608
Name: 37, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 65:
banos    1
Name: 65, dtype: object
baños_completos    0.0
Name: 65, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-c7136c4c8f36-cb2f-fd601220-8b1e-4305
Name: 65, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manu

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10972\1369790780.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'banos'] = df.at[idx, 'baños_completos']



---

Registro 6993:
banos    1
Name: 6993, dtype: object
baños_completos    0.0
Name: 6993, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-eb0c79af90fc-430-e6bbff06-a0f7-33ad
Name: 6993, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 7019:
banos    1
Name: 7019, dtype: object
baños_completos    0.0
Name: 7019, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-50efc0ce8b46-81d9-289506e0-957b-402a
Name: 7019, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'banos' por el de 'baños_completos'
3. Pasar al siguiente registro
Registro 7020:
banos    1
Name: 7020, dtype: object
baños_completos    0.0
Name: 7020, dtype: object
url    https://www.lamudi.com.mx/detalle/41032-73-b89ad56639c5-4da8-7f8d22f8-99a6-3892
Name: 7020, dtype: object

¿Qué acción te gustaría realiz

### Medio baños

#### [0 / 0]

Poner como mínimo para el registro 1 en baños, pero si cuenta con valor en descripción que se le asigne ese valor.

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[(df['medio_baños'] == 0) & (df['medios_baños'].isna())]
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    #print("2. Asignar 1 como mínimo y pasar al siguiente registro")
    print("2. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'banos': ")
        df.at[idx, 'banos'] = float(new_value)
    #elif option == '2':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'banos'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

KeyError: 'medio_baños'

#### [0 / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( (df['medio_baños']==0) & (df['medios_baños'].notna()) )] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'medio_baños' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_baños': ")
        df.at[idx, 'medio_baños'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_baños'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

#### [# / #]

In [ ]:
# Filtrar los registros según las condiciones dadas
totales = df[( ((df['medio_baños']!=0) & (df['medios_baños'].notna())) & (df['medio_baños'] != df['medios_baños']))] 
pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in totales.iterrows():
    print(f"Registro {idx}:")
    print(row[['url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'medio_baños' por el de 'medios_baños'")
    #print("3. Asignar 1 como mínimo y pasar al siguiente registro")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1 | 2 | 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'medio_baños': ")
        df.at[idx, 'medio_baños'] = float(new_value)
    elif option == '2':
        # Sustituir el valor de 'medio_baños' por 'medios_baños'
        df.at[idx, 'medio_baños'] = df.at[idx, 'medios_baños']
    #elif option == '3':
        # Asignar valor de 1 como mínimo
        #df.at[idx, 'medio_baños'] = 1
        #continue  # Pasar al siguiente registro
    else:
        continue  # Pasar al siguiente registro

    print("\n---\n")

print("Proceso completado.")

### B) Total de baños

In [ ]:
# Make the medio_baños and banos sum
# 'medio_baños', divididos entre 2 para representar medios
df['bano_total'] = df['banos'] + (df['medio_banos'] / 2)
df['bano_total'] = df['bano_total'].round(1)  # Round the total, not just 'banos'
print(df[['banos', 'medio_banos', 'bano_total']].head())

   banos  medio_banos  bano_total
0      2            1         2.5
1      3            1         3.5
2      2            1         2.5
3      1            0         1.0
4      3            0         3.0


# 10. Ubicación

## a) Desde descripción
#### Todavía en espera de estructura de código

In [8]:
# Función para extraer ubicación de la descripción
def extraer_ubicacion(descripcion):
    descripcion = descripcion.lower()
    # Patrones para buscar ubicaciones comunes
    patrones = [
        r'(?:ubicad[oa] en|en) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:calle|avenida|av\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:colonia|col\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})',
        r'(?:fraccionamiento|fracc\.) ([\w\s]+(?:,\s*[\w\s]+){0,2})'
    ]
    
    for patron in patrones:
        match = re.search(patron, descripcion)
        if match:
            return match.group(1).strip().title()
    
    return None

# Aplicar la función al DataFrame
df['ubicacion_desc'] = df['descripcion'].apply(extraer_ubicacion)

In [9]:
df['ubicacion_desc'].unique()

array(['Gran Reserva Juriquilla',
       'La Colonia Cimatario Muy Amplia En Una Excelente Colonia De Queretaro Dentro De Fraccionamiento Privado Con Vigilancia Las 24 Hrs Y Muy Centrica Con Acceso A Avenidas Principales Muy Cerca De Centros Comerciales, Escuelas Y Hospitales, La Casa Cuenta Con',
       'Fraccionamiento Privado En Av', ..., 'Todos Los Espacios',
       'Colinas Del Cimatario 535 M2 De Terreno Y 534 M2 De Construcción, Con Excelente Ubicación A Una Cuadra De Boulevard Centro Sur',
       'Varios Niveles En \xa0Esquina Col'], dtype=object)

#### Cuando la ubicación dice centro:

In [28]:
# Filtrar los registros donde la ubicación contiene la palabra "centro"
df_centro = df[df['ubicacion'].str.contains('centro', case=False, na=False)]
num_registros_centro = len(df_centro)

print(f"Número de registros que contienen 'centro' en la ubicación: {num_registros_centro}")

Número de registros que contienen 'centro' en la ubicación: 823


In [26]:

pd.set_option('display.max_colwidth', None) # Mostrar URLs completas

# Iterar sobre los registros filtrados
for idx, row in df_centro.iterrows():
    print(f"Registro {idx}:")
    print(row[['ubicacion', 'ubicacion_desc', 'url']])
    print("\n¿Qué acción te gustaría realizar?")
    print("1. Sustituir el valor manualmente")
    print("2. Sustituir el valor de 'ubicacion' por el de 'ubicacion_desc'")
    print("3. Pasar al siguiente registro")

    # Solicitar la opción al usuario
    option = input("Elige una opción (1, 2, 3): ")

    if option == '1':
        # Sustituir el valor manualmente
        new_value = input("Introduce el nuevo valor para 'ubicacion': ")
        df.at[idx, 'ubicacion'] = new_value  
    elif option == '2':
        # Sustituir el valor de 'ubicacion' por 'ubicacion_desc'
        df.at[idx, 'ubicacion'] = df.at[idx, 'ubicacion_desc']
    elif option == '3':
        # Pasar al siguiente registro
        continue
    else:
        print("Opción no válida. Pasando al siguiente registro...")
    
    print("\n---\n")

print("Proceso completado.")


Registro 5:
ubicacion                                          Residencial Centro Sur Condominio I, Querétaro, Querétaro
ubicacion_desc    Cuanto A La Propiedad En Sí, Cuenta Con Todo Lo Que Necesitas Y Más Para Vivir Cómodamente
url                         https://www.lamudi.com.mx//detalle/41032-73-5b038591d2b2-2867-4a0a3194-ae81-4856
Name: 5, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'ubicacion' por el de 'ubicacion_desc'
3. Pasar al siguiente registro



---

Registro 28:
ubicacion                                                   Querétaro Centro, Querétaro, Querétaro
ubicacion_desc                                                                                None
url               https://www.lamudi.com.mx//detalle/41032-73-17a1b1c74088-3e68-be5ac63a-b459-4288
Name: 28, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'ubicacion' por el de 'ubicacion_desc'
3. Pasar al siguiente registro

---

Registro 117:
ubicacion                                                   Querétaro Centro, Querétaro, Querétaro
ubicacion_desc                            Bienes Raíces De Kioto Service Y Asegura Tu Futuro Hogar
url               https://www.lamudi.com.mx//detalle/41032-73-e1bcd19eec35-4036-b7fad8f4-b9a1-4f12
Name: 117, dtype: object

¿Qué acción te gustaría realizar?
1. Sustituir el valor manualmente
2. Sustituir el valor de 'ubicacion' por el de 'ubicacion_desc'
3. Pasar al siguiente

## b) Separación

Se divide la ubicación por 
* Colonia
* Municipio
* Estado

In [27]:
# Dividir 'ubicacion' en un máximo de 3 partes y manejar valores faltantes
df[['Colonia', 'Municipio', 'Estado']] = df['ubicacion'].str.split(',', expand=True, n=2)

# Limpiar espacios alrededor de los valores
df['Colonia'] = df['Colonia'].str.strip()
df['Municipio'] = df['Municipio'].str.strip()
df['Estado'] = df['Estado'].str.strip()

# Rellenar valores NaN o faltantes con valores predeterminados específicos
df['Colonia'] = df['Colonia'].fillna('Sin_colonia')
df['Municipio'] = df['Municipio'].fillna('Sin_municipio')
df['Estado'] = df['Estado'].fillna('Sin_estado')
df[['ubicacion', 'Colonia', 'Municipio', 'Estado']]

,ubicacion,Colonia,Municipio,Estado
0,"Fraccionamiento San Isidro, Querétaro, Querétaro",Fraccionamiento San Isidro,Querétaro,Querétaro
1,"Cimatario, Querétaro, Querétaro",Cimatario,Querétaro,Querétaro
2,"El Refugio, Querétaro, Querétaro",El Refugio,Querétaro,Querétaro
3,"Andadores Satélite, Querétaro, Querétaro",Andadores Satélite,Querétaro,Querétaro
4,"Fraccionamiento Juriquilla Grand, Querétaro, Querétaro",Fraccionamiento Juriquilla Grand,Querétaro,Querétaro
...,...,...,...,...
9339,"Fraccionamiento El Campanario, Querétaro, Querétaro",Fraccionamiento El Campanario,Querétaro,Querétaro
9340,"Fraccionamiento El Campanario, Querétaro, Querétaro",Fraccionamiento El Campanario,Querétaro,Querétaro
9341,"Fraccionamiento El Campanario, Querétaro, Querétaro",Fraccionamiento El Campanario,Querétaro,Querétaro
9342,"Centro Sur, Querétaro, Querétaro",Centro Sur,Querétaro,Querétaro


#### Separación de tipo de lugar del nombre de la colonia

In [29]:
tipos = ['Condominio', 'Fraccionamiento', 'Residencial', 'Colonia']
def obtener_tipo_lugar(nombre):
    for tipo in tipos:
        if tipo in nombre:
            return tipo
    return None

df['TIPO_LUGAR'] = df['Colonia'].apply(obtener_tipo_lugar)
# Eliminar el tipo de lugar de la columna 'colonia'
df['Colonia'] = df.apply(lambda row: row['Colonia'].replace(row['TIPO_LUGAR'], '').strip() if row['TIPO_LUGAR'] else row['Colonia'], axis=1)
print(df[['Colonia','TIPO_LUGAR','ubicacion']])

                 Colonia       TIPO_LUGAR  \
0             San Isidro  Fraccionamiento   
1              Cimatario             None   
2             El Refugio             None   
3     Andadores Satélite             None   
4       Juriquilla Grand  Fraccionamiento   
...                  ...              ...   
9339       El Campanario  Fraccionamiento   
9340       El Campanario  Fraccionamiento   
9341       El Campanario  Fraccionamiento   
9342          Centro Sur             None   
9343        Niños Héroes             None   

                                                   ubicacion  
0           Fraccionamiento San Isidro, Querétaro, Querétaro  
1                            Cimatario, Querétaro, Querétaro  
2                           El Refugio, Querétaro, Querétaro  
3                   Andadores Satélite, Querétaro, Querétaro  
4     Fraccionamiento Juriquilla Grand, Querétaro, Querétaro  
...                                                      ...  
9339     Fracciona

## c) Estandarizar valores Colonia

In [30]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['Colonia'] = df['Colonia'].astype(str)
# Convertir a minúsculas
df['Colonia'] = df['Colonia'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')

# Aplicar la función de eliminación de acentos
df['Colonia'] = df['Colonia'].apply(eliminar_acentos)

# Reemplazar Ñ
df['Colonia'] = df['Colonia'].str.replace('ñ', 'n')
df['Colonia'] = df['Colonia'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

## d) Mapeo de CP y tipo de lugar por diccionario

In [31]:
# Cargar todas las hojas del archivo Excel en un diccionario de DataFrames
ruta_excel = 'D:\yoe11\Documents\TESEO\Modelo Factible\Scraplining\oficial\cp_dictionary.xlsx'  
hojas = pd.read_excel(ruta_excel, sheet_name=None)

# Obtener combinaciones únicas de Estado, Municipio y Colonia
combinaciones_unicas = df[['Estado', 'Municipio', 'Colonia']].drop_duplicates()

# Crear un diccionario para almacenar los códigos postales y tipos de asentamiento encontrados
codigos_postales = {}
tipos_asentamiento = {}

# Buscar el código postal y tipo de asentamiento para cada combinación única
for _, row in combinaciones_unicas.iterrows():
    estado = row['Estado'].strip()
    municipio = row['Municipio'].strip()
    colonia = row['Colonia'].strip()

    # Verificar si el estado existe en las hojas del Excel
    if estado in hojas:
        df_estado = hojas[estado]

        # Filtrar por municipio y colonia
        df_filtrado = df_estado[
            (df_estado['D_mnpio'].str.strip().str.lower() == municipio.lower()) &
            (df_estado['d_asenta'].str.strip().str.lower() == colonia.lower())
        ]

        # Si hay coincidencias, almacenar el primer código postal y tipo de asentamiento en los diccionarios
        if not df_filtrado.empty:
            codigos_postales[(estado, municipio, colonia)] = df_filtrado['d_codigo'].iloc[0]
            tipos_asentamiento[(estado, municipio, colonia)] = df_filtrado['d_tipo_asenta'].iloc[0]
        else:
            codigos_postales[(estado, municipio, colonia)] = None
            tipos_asentamiento[(estado, municipio, colonia)] = None
    else:
        codigos_postales[(estado, municipio, colonia)] = None
        tipos_asentamiento[(estado, municipio, colonia)] = None

# Crear columnas 'Codigo_Postal' y 'Tipo_Asentamiento' en el DataFrame original usando los diccionarios
df['Codigo_Postal'] = df.apply(lambda x: codigos_postales.get((x['Estado'].strip(), x['Municipio'].strip(), x['Colonia'].strip())), axis=1)
df['Tipo_Asentamiento'] = df.apply(lambda x: tipos_asentamiento.get((x['Estado'].strip(), x['Municipio'].strip(), x['Colonia'].strip())), axis=1)

# Obtener las combinaciones únicas de las columnas 'Colonia', 'Municipio', 'Estado', 'Codigo_Postal', 'Tipo_Asentamiento'
combinaciones_unicas = df[['Colonia', 'Municipio', 'Estado', 'Codigo_Postal', 'Tipo_Asentamiento']].drop_duplicates()
for idx, row in combinaciones_unicas.iterrows():
    print(f"Combinación {idx + 1}:")
    print(f"Colonia: {row['Colonia']}, Municipio: {row['Municipio']}, Estado: {row['Estado']}, Codigo_Postal: {row['Codigo_Postal']}, Tipo_Asentamiento: {row['Tipo_Asentamiento']}")
    print("---")


Combinación 1:
Colonia: san isidro, Municipio: Querétaro, Estado: Querétaro, Codigo_Postal: 76226.0, Tipo_Asentamiento: Fraccionamiento
---
Combinación 2:
Colonia: cimatario, Municipio: Querétaro, Estado: Querétaro, Codigo_Postal: 76030.0, Tipo_Asentamiento: Colonia
---
Combinación 3:
Colonia: el refugio, Municipio: Querétaro, Estado: Querétaro, Codigo_Postal: nan, Tipo_Asentamiento: None
---
Combinación 4:
Colonia: andadores satelite, Municipio: Querétaro, Estado: Querétaro, Codigo_Postal: nan, Tipo_Asentamiento: None
---
Combinación 5:
Colonia: juriquilla grand, Municipio: Querétaro, Estado: Querétaro, Codigo_Postal: nan, Tipo_Asentamiento: None
---
Combinación 6:
Colonia: el mirador, Municipio: El Marqués, Estado: Querétaro Arteaga, Codigo_Postal: nan, Tipo_Asentamiento: None
---
Combinación 7:
Colonia: parque santiago, Municipio: Querétaro, Estado: Querétaro, Codigo_Postal: 76115.0, Tipo_Asentamiento: Fraccionamiento
---
Combinación 8:
Colonia: felipe carrillo puerto, Municipio: Qu

____
# 11. Variables oficiales

## a) Precios

In [ ]:
df[['precio','denominacion','precio_desc','precio_mxn','precio_usd','fecha_conversion']]

In [ ]:
df = df.drop(['denominacion', 'precio_desc',], axis=1, errors='ignore') #'fecha_conversion'

## b) Baños

In [ ]:
df[['banos','medio_banos','baños_desc','baños_completos','medios_baños','bano_total']]

In [ ]:
df = df.drop(['baños_desc','baños_completos','medios_baños'], axis=1, errors='ignore')

# 12. Estandarizar

#### Propiedad

In [ ]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['propiedad'] = df['propiedad'].astype(str)
# Convertir a minúsculas
df['propiedad'] = df['propiedad'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['propiedad'] = df['propiedad'].apply(eliminar_acentos)

# Reemplazar Ñ
df['propiedad'] = df['propiedad'].str.replace('ñ', 'n')
df['propiedad'] = df['propiedad'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

#### Ubicación

In [ ]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['ubicacion'] = df['ubicacion'].astype(str)
# Convertir a minúsculas
df['ubicacion'] = df['ubicacion'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['ubicacion'] = df['ubicacion'].apply(eliminar_acentos)

# Reemplazar Ñ
df['ubicacion'] = df['ubicacion'].str.replace('ñ', 'n')
df['ubicacion'] = df['ubicacion'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

#### Descripción

In [ ]:
# Convierte todos los valores a strings, maneja NaN o None como cadenas vacías
df['descripcion'] = df['descripcion'].astype(str)
# Convertir a minúsculas
df['descripcion'] = df['descripcion'].str.lower()

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['descripcion'] = df['descripcion'].apply(eliminar_acentos)

# Reemplazar Ñ
df['descripcion'] = df['descripcion'].str.replace('ñ', 'n')
df['descripcion'] = df['descripcion'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

# 13. Exportar

LAS VARIABLES CON LAS QUE SE ESPERA TENER PARA SQL SON:
* id
* categoria
* propiedad
* precio
* tipo
* personas_interesadas
* m2_total (metros_total)
* m2_contruido
* precio_m2_terreno
* precio_m2_contruido
* promedio
* baño
* medio_baño
* baño_total
* estacionamiento
* recamaras
* antiguedad
* publicado_hace
* status(Estado)

In [33]:
df.columns

Index(['precio', 'denominacion', 'propiedad', 'metros_total',
       'metros_construido', 'tiempo_de_publicacion', 'tipo',
       'estacionamientos', 'recamaras', 'banos', 'medio_banos',
       'seguridad_privada', 'ubicacion', 'url', 'descripcion',
       'ubicacion_desc', 'Colonia', 'Municipio', 'Estado', 'TIPO_LUGAR',
       'Codigo_Postal', 'Tipo_Asentamiento'],
      dtype='object')

In [ ]:
df['id'] = range(1, len(df) + 1)
df.to_csv('mayo_queretaro_2024.csv', encoding='utf-8', index=False)